In [13]:
import os
import pickle
import pandas as pd

In [5]:
os.getcwd()

'/home/jonasmmiguel/Documents/learning/poli/thesis/wind-stf/prototyping'

# define CV splits

In [16]:
df_infer = capacity_factors_daily_2000to2015 = pd.read_hdf(
    path_or_buf='../data/05_model_input/df_infer.hdf', 
    key='df_infer'
)

df_infer.head(3)

,DE145,DE114,DE146,DE132,DE12A,DE133,DE12C,DE11C,DE118,DE119,...,DEG01,DEG0F,DE275,DE21C,DE234,DE251,DE276,DE278,DE718,DE943
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,0.219507,0.269458,0.240540,0.263019,0.593148,0.331149,0.555176,0.284140,0.357407,0.410291,...,0.300818,0.363388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-02,0.083655,0.063571,0.043303,0.005984,0.080731,0.007986,0.066120,0.083709,0.078740,0.108224,...,0.150699,0.133930,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-03,0.246707,0.229298,0.337413,0.047907,0.279092,0.136004,0.237430,0.320095,0.460150,0.516133,...,0.348737,0.297686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [166]:
cv_params = {
    'method': 'expanding_windows',
    'relsize_shortest_train_window': 0.6,
     # 'train_window_relsize_last_pass': 1.0,  # always considered to be 1.0 
    'n_passes': 3,
    'steps_ahead': 1,
    'size_forecast_window': 7,
}

In [167]:
cv_method = cv_params['method']
n_passes = cv_params['n_passes']
relsize_shortest_train_window = cv_params['relsize_shortest_train_window']
size_forecasting_window = cv_params['size_forecast_window']
steps_ahead = cv_params['steps_ahead']

In [164]:
len(df_infer) 

903

In [168]:
train_end_idx + forecasting_window_size

903

In [169]:
import numpy as np

In [173]:
cv_splits = {}

# max train size so that train  + gap (steps_ahead) + val still fit in inference data
relsize_longest_train_window = 1 - ((steps_ahead - 1) + size_forecasting_window) / len(df_infer)

window_relsize = np.linspace(
    start=relsize_shortest_train_window, 
    stop=relsize_longest_train_window, 
    num=n_passes
)

for p in range(n_passes):
    pass_id = 'pass ' + str(p + 1)
    
    train_end_idx = round(window_relsize[p] * len(df_infer))    

    cv_splits[pass_id] = {
        'train': slice(0, train_end_idx),
        'val': slice(train_end_idx, train_end_idx + size_forecasting_window)
    }  

In [171]:
cv_splits

{'pass 1': {'train': slice(0, 542, None), 'val': slice(542, 549, None)},
 'pass 2': {'train': slice(0, 719, None), 'val': slice(719, 726, None)},
 'pass 3': {'train': slice(0, 896, None), 'val': slice(896, 903, None)}}

In [172]:
df_infer[ cv_splits['pass 3']['val'] ].index

DatetimeIndex(['2015-06-16', '2015-06-17', '2015-06-18', '2015-06-19',
               '2015-06-20', '2015-06-21', '2015-06-22'],
              dtype='datetime64[ns]', name='date', freq='D')